In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
import torch.nn as nn

# Construct customized ResNet

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, pcpt_block, pcpt_layers, scoop_block, scoop_layers, h, w, pcpt_is_upsample=0, scoop_is_upsample=0):
        self.inplanes = 64
        self.pcpt_is_upsample = pcpt_is_upsample
        super(ResNet, self).__init__()
        self.pcpt_conv1 = nn.Conv2d(4, 64, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.pcpt_bn1 = nn.BatchNorm2d(64)
        self.pcpt_relu = nn.ReLU(inplace=True)
        self.pcpt_maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.pcpt_upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        self.pcpt_layer1 = self._make_layer(pcpt_block, 128, pcpt_layers[0])
        self.pcpt_layer2 = self._make_layer(pcpt_block, 256, pcpt_layers[1])
        self.pcpt_layer3 = self._make_layer(pcpt_block, 512, pcpt_layers[2])

        self.inplanes = 512
        self.scoop_is_upsample = scoop_is_upsample
        self.scoop_upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        self.scoop_layer1 = self._make_layer(scoop_block, 256, scoop_layers[0])
        self.scoop_layer2 = self._make_layer(scoop_block, 128, scoop_layers[1])
        self.scoop_layer3 = self._make_layer(scoop_block, 64, scoop_layers[2])
        self.scoop_conv1 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.scoop_bn1 = nn.BatchNorm2d(1)
        self.scoop_conv2 = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.scoop_bn2 = nn.BatchNorm2d(3)
        self.scoop_relu = nn.ReLU(inplace=True)

        self.scoop_head = nn.Linear(h*w, 1)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.pcpt_conv1(x)
        x = self.pcpt_bn1(x)
        x = self.pcpt_relu(x)
        x = self.pcpt_maxpool(x)

        x = self.pcpt_layer1(x)
        x = self.pcpt_maxpool(x)
        x = self.pcpt_layer2(x)
        x = self.pcpt_layer3(x)

        x = self.scoop_layer1(x)
        x = self.scoop_layer2(x)
        x = self.scoop_upsample(x)
        x = self.scoop_layer3(x)
        x = self.scoop_upsample(x)

        x = self.scoop_conv2(x)
        x = self.scoop_bn2(x)
        x = self.scoop_relu(x)

        return x

In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.utils.data as Data
import scipy.ndimage
BATCH_SIZE =  32
NUM_EPOCH = 50

# make data

data_input = np.load("data_20210715/input_data_array_20210715.npy")/255.0
#normalization=np.ones(data_input.shape)
#normalization[:,:,:]=[255,255,255,0.08]
#data_input = data_input/normalization
#print(data_input)
print(data_input.shape)

(22124, 200, 200, 4)


In [4]:
import random
'''
input_array_mean = np.mean(np.mean(np.mean(data_input, axis=0), axis=0), axis=0)
input_array_std = np.mean(np.mean(np.std(data_input, axis=0), axis=0), axis=0)
print('input_array_mean', input_array_mean)
print('input_array_std', input_array_std)
'''

#input_array_mean=np.ones(data_input.shape)
#input_array_mean[:,:,:]=[0.54569177,0.48216905,0.49853667,0.42829879]
#input_array_std=np.ones(data_input.shape)
#input_array_std[:,:,:]=[0.31950833,0.29141234,0.30389949,0.20277719]

#data_input = (data_input-input_array_mean)/input_array_std

temp_index_set = random.sample(range(len(data_input)), 50)
#print(temp_index_set)

temp_index_set_other = list(set(range(len(data_input))).difference(set(temp_index_set)))
#print(temp_index_set_other)

In [5]:
import gc
data_input = torch.from_numpy(data_input).permute(0,3,1,2)

train_data_input = data_input[temp_index_set_other, :, :, :]
val_data_input = data_input[temp_index_set, :, :, :]
htmap_h = data_input.shape[2]
htmap_w = data_input.shape[3]
del data_input
gc.collect()

20

In [6]:
import gc
data_label = np.load("data_20210715/label_data_array_20210715.npy")
print(data_label.shape)
data_label[data_label==0]=2
data_label[data_label==255]=0
data_label[(data_label==128)]=1
#print(data_label[np.logical_and(np.logical_and((data_label!=0),(data_label!=1)),(data_label!=2))])


good_cnt = (data_label==1).sum()
bad_cnt = (data_label==2).sum()
print(good_cnt, bad_cnt)
total = good_cnt + bad_cnt

weights = [0, total/good_cnt, total/bad_cnt]
class_weights = torch.FloatTensor(weights)
if torch.cuda.is_available():
    class_weights = class_weights.cuda()
print('Class weights:', class_weights)

train_data_label = data_label[temp_index_set_other, :, :]
print('step1')
val_data_label = data_label[temp_index_set, :, :]
del data_label
gc.collect()
print('step2')
print(train_data_label.shape, train_data_label.dtype)
print(val_data_label.shape, val_data_label.dtype)
train_data_label = torch.from_numpy(train_data_label)
val_data_label = torch.from_numpy(val_data_label)

print('step3')
train_data_label = train_data_label.long()
val_data_label = val_data_label.long()

(22124, 200, 200)
517031 262824989
Class weights: tensor([  0.0000, 509.3351,   1.0020], device='cuda:0')
step1
step2
(22074, 200, 200) uint8
(50, 200, 200) uint8
step3


In [7]:
train_torch_dataset = Data.TensorDataset(train_data_input, train_data_label)
print('step4')
loader = Data.DataLoader(dataset=train_torch_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('step5')

step4
step5


In [8]:
net = ResNet(pcpt_block=BasicBlock, pcpt_layers=[1,5,1], scoop_block=BasicBlock, scoop_layers=[1,5,1], h=htmap_h, w=htmap_w).cuda()     # define the network
print(net)  # net architecture

ResNet(
  (pcpt_conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (pcpt_bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pcpt_relu): ReLU(inplace=True)
  (pcpt_maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pcpt_upsample): Upsample(scale_factor=2.0, mode=bilinear)
  (pcpt_layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum

In [ ]:
#optimizer = torch.optim.SGD(params = net.parameters(), lr=1e-4, momentum=0.9, weight_decay=2e-5)
optimizer = torch.optim.Adam(params = net.parameters(), lr=1e-4)
#optimizer = torch.optim.SGD(params = net.parameters(), lr=1e-4)
loss_func = torch.nn.CrossEntropyLoss(weight=class_weights).cuda()  # the target label is NOT an one-hotted

val_data_input = val_data_input.cuda().float()
val_data_label = val_data_label.cuda()

min_val_loss_print=float('inf')
for epoch in range(NUM_EPOCH):
    for step, (batch_data, batch_label) in enumerate(loader):
        batch_data = batch_data.cuda().float()
        batch_label = batch_label.cuda()
        net.train()
        output  = net(batch_data)
        train_loss = loss_func(output,batch_label)
        optimizer.zero_grad()
        train_loss.backward()
        #nn.utils.clip_grad_norm_(net.parameters(), max_norm=20, norm_type=2)
        optimizer.step()
        train_loss_print = train_loss.data.item()
        with torch.no_grad():
            net.eval()
            val_output  = net(val_data_input)
            val_loss = loss_func(val_output,val_data_label)
            val_loss_print = val_loss.data.item()
            torch.cuda.empty_cache()
        #print('epoch: ', epoch, '  step: ', step, '  train loss: ', train_loss_print, ' val loss: ', val_loss_print)
        print('epoch: ', epoch, '  step: ', step, '  train loss: ', train_loss_print, ' val loss: ', val_loss_print)
        if val_loss_print < min_val_loss_print:
            torch.save(net.state_dict(), 'net_params_20210715/net_params_20210715_1/epoch_'+str(epoch)+'.pkl') 
            min_val_loss_print = val_loss_print
            print('min_val_loss_print', min_val_loss_print)

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


epoch:  0   step:  0   train loss:  1.1575491428375244  val loss:  1.719327688217163
min_val_loss_print 1.719327688217163
epoch:  0   step:  1   train loss:  1.176862120628357  val loss:  1.4551018476486206
min_val_loss_print 1.4551018476486206
epoch:  0   step:  2   train loss:  0.9605910778045654  val loss:  1.293009877204895
min_val_loss_print 1.293009877204895
epoch:  0   step:  3   train loss:  1.0098267793655396  val loss:  1.3510037660598755
epoch:  0   step:  4   train loss:  1.1145790815353394  val loss:  1.3881511688232422
epoch:  0   step:  5   train loss:  1.03610098361969  val loss:  1.3610345125198364
epoch:  0   step:  6   train loss:  1.0199413299560547  val loss:  1.310666799545288
epoch:  0   step:  7   train loss:  0.9649294018745422  val loss:  1.297982096672058
epoch:  0   step:  8   train loss:  0.9855425953865051  val loss:  1.2694835662841797
min_val_loss_print 1.2694835662841797
epoch:  0   step:  9   train loss:  0.88344806432724  val loss:  1.2344201803207397

In [1]:
import numpy as np
import torch
a=torch.from_numpy((np.random.rand(22074, 200, 200)*200).astype(np.uint8))
print(a)
np.array([1]).astype(np.uint8)
print(np.array([1]).astype(np.uint8).dtype)

tensor([[[114, 196, 182,  ...,  75, 156, 108],
         [ 32, 105, 147,  ...,  99,  10, 143],
         [ 11, 194, 140,  ..., 126,  23, 108],
         ...,
         [163, 118,  47,  ..., 166,  67, 167],
         [195, 176, 169,  ...,  18,  43, 196],
         [114,  50, 182,  ..., 159, 151,  74]],

        [[109, 194, 197,  ..., 132,  58,  60],
         [157,   1, 118,  ...,   9,  46,  54],
         [152, 128, 137,  ...,  79, 196, 145],
         ...,
         [162,  62, 152,  ...,   7,   9, 147],
         [191,  96,  47,  ...,  72, 183, 149],
         [110, 186, 141,  ..., 119, 109, 162]],

        [[166, 150,  87,  ..., 148, 167,  38],
         [ 68,  58, 115,  ...,  66,  38,  73],
         [ 39,  44,  89,  ...,   8, 175, 108],
         ...,
         [110, 153,  64,  ..., 161, 158,  68],
         [120, 142,  72,  ..., 161, 186,  80],
         [148, 190, 182,  ...,  23,  30,   6]],

        ...,

        [[ 91,  52, 122,  ...,  20, 132, 165],
         [161,  78, 197,  ...,  10, 100, 103]

In [5]:
print(train_data_label)

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         ...,
         [2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2]],

        [[2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         ...,
         [2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2]],

        ...,

        [[2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         ...,
         [2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,